# Introduction to BNNs with Larq

<a href="https://colab.research.google.com/github/larq/docs/blob/master/docs/larq/tutorials/mnist.ipynb"><button class="notebook-badge">Run on Colab</button></a> <a href="https://github.com/larq/docs/blob/master/docs/larq/tutorials/mnist.ipynb"><button class="notebook-badge">View on GitHub</button></a>

This tutorial demonstrates how to train a simple binarized Convolutional Neural Network (CNN) to classify MNIST digits. This simple network will achieve approximately 98% accuracy on the MNIST test set. This tutorial uses Larq and the [Keras Sequential API](https://www.tensorflow.org/guide/keras), so creating and training our model will require only a few lines of code.

In [ ]:
pip install larq

In [ ]:
import tensorflow as tf
import larq as lq

### Download and prepare the MNIST dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

### Create the model

The following will create a simple binarized CNN.

The quantization function
$$
q(x) = \begin{cases}
    -1 & x < 0 \\\
    1 & x \geq 0
\end{cases}
$$
is used in the forward pass to binarize the activations and the latent full precision weights. The gradient of this function is zero almost everywhere which prevents the model from learning.

To be able to train the model the gradient is instead estimated using the Straight-Through Estimator (STE)
(the binarization is essentially replaced by a clipped identity on the backward pass):
$$
\frac{\partial q(x)}{\partial x} = \begin{cases}
    1 & \left|x\right| \leq 1 \\\
    0 & \left|x\right| > 1
\end{cases}
$$

In Larq this can be done by using `input_quantizer="ste_sign"` and `kernel_quantizer="ste_sign"`.
Additionally, the latent full precision weights are clipped to -1 and 1 using `kernel_constraint="weight_clip"`.

In [ ]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")

model = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
model.add(lq.layers.QuantConv2D(32, (3, 3),
                                kernel_quantizer="ste_sign",
                                kernel_constraint="weight_clip",
                                use_bias=False,
                                input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())

model.add(lq.layers.QuantDense(64, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantDense(10, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

Almost all parameters in the network are binarized, so either -1 or 1. This makes the network extremely fast if it would be deployed on custom BNN hardware.

 Here is the complete architecture of our model:

In [ ]:
lq.models.summary(model)

### Compile and train the model

Note: This may take a few minutes depending on your system.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history=model.fit(train_images, train_labels, batch_size=64, epochs=6)

test_loss, test_acc = model.evaluate(test_images, test_labels)

### Evaluate the model

In [ ]:
print(f"Test accuracy {test_acc * 100:.2f} %")

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
#plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# 遍历模型的每一层
for i, layer in enumerate(model.layers):
    # 检查是否是 QuantConv2D 层
    if isinstance(layer, lq.layers.QuantConv2D):
        # 提取权重
        weights = layer.get_weights()

        # 打印层索引和权重形状
        print(f"Layer {i} ({layer.name}) weights shape: {weights[0].shape}")

        # 打印权重值
        print("Weights:\n", weights[0])

In [ ]:
import larq as lq

# 遍历模型的每一层
for i, layer in enumerate(model.layers):
    # 检查是否是 QuantConv2D 层
    if isinstance(layer, lq.layers.QuantConv2D):
        # 提取浮点数权重
        weights = layer.get_weights()[0]

        # 使用 ste_sign 对权重进行量化
        quantized_weights = lq.quantizers.ste_sign(weights)

        # 打印量化后的权重
        print(f"Layer {i} ({layer.name}) quantized weights shape: {quantized_weights.shape}")
        print("Quantized weights:\n", quantized_weights)

In [ ]:
import tensorflow as tf

# 假设输入数据为 X_test 的第一张图片
input_data = test_images[0:1]  # 形状为 (1, 28, 28, 1)

# 逐层计算输出
output = input_data
for i, layer in enumerate(model.layers):
    # 如果是 QuantConv2D 层，使用量化后的权重
    if isinstance(layer, lq.layers.QuantConv2D):
        weights = layer.get_weights()[0]
        quantized_weights = lq.quantizers.ste_sign(weights)

        # 使用量化后的权重进行卷积
        output = tf.nn.conv2d(output, quantized_weights, strides=layer.strides, padding=layer.padding.upper())

    # 如果是 MaxPooling2D 层，进行最大池化
    elif isinstance(layer, tf.keras.layers.MaxPooling2D):
        output = tf.nn.max_pool2d(output, ksize=layer.pool_size, strides=layer.strides, padding=layer.padding.upper())

    # 如果是 BatchNormalization 层，进行批量归一化
    elif isinstance(layer, tf.keras.layers.BatchNormalization):
        output = tf.nn.batch_normalization(output, layer.moving_mean, layer.moving_variance, layer.beta, layer.gamma, layer.epsilon)

    # 如果是 QuantDense 层，使用量化后的权重
    elif isinstance(layer, lq.layers.QuantDense):
        weights = layer.get_weights()[0]
        quantized_weights = lq.quantizers.ste_sign(weights)

        # 使用量化后的权重进行矩阵乘法
        output = tf.matmul(output, quantized_weights)

    # 如果是激活函数层，应用激活函数
    elif isinstance(layer, tf.keras.layers.Activation):
        output = layer.activation(output)

    # 打印每一层的输出形状
    print(f"Layer {i} ({layer.name}) output shape: {output.shape}")

# 打印最终输出
print("Final output:\n", output)

As you can see, our simple binarized CNN has achieved a test accuracy of around 98 %. Not bad for a few lines of code!

For information on converting Larq models to an optimized format and using or benchmarking them on Android or ARM devices, have a look at [this guide](https://docs.larq.dev/compute-engine/end_to_end/).